## TaskWeaver

This notebook aims to condence all requirements needed for coding up a taskweaver project.

Firstly, you need to make a project folder. this is a prerequisite to running TaskWeaver. 

Then we need to make a session:
> TaskWeaver supports a basic session management to keep different users' <br>
> data separate. The code execution is separated into different processes <br>
> in order not to interfere with each other.

See [starter_project]("../_starter_projects/")

We also need to enable modules that the code interpreter can use and code verification.
> Code verification - TaskWeaver is designed to verify the generated code before execution. <br>
> It can detect potential issues in the generated code and provide suggestions to fix them.

Add this to the `taskweaver_config.json`

```
"code_verification.allowed_modules": ["pandas", "matplotlib", "numpy", "sklearn", "scipy", "seaborn", "datetime", "typing"],
"code_verification.code_verification_on": true,
```

In [1]:
import json
from typing import Any, Optional

import pprint as pp
from taskweaver.app.app import TaskWeaverApp

# Create a session
# would prefer if the expected folder structure was autogenerated
app_dir = "../_starter_projects/"
app = TaskWeaverApp(app_dir=app_dir)
session = app.get_session()

### Configuration

In [2]:
# you can add customer configurations programatically by adding
# something like the following to taskweaver_config.json
# "session.max_internal_chat_round_num": 5
# or  session.config.max_internal_chat_round_num = 5
session.config.max_internal_chat_round_num

15

In [3]:
session.config.max_internal_chat_round_num = 30
session.config.max_internal_chat_round_num

30

In [4]:
# Here is where the configurations from taskweaver_config.json are stored
# https://github.com/microsoft/TaskWeaver/blob/main/docs/configuration.md 
# Categories of keys within the config
# - session_manager
# - llm
# - workspace
# - logging
# - session
# - planner
# - plugin
# - round_compressor
# - code_generator
# - embedding_model
# - code_verification
# - code_interpreter
session.config.src.config


{'session_manager.store_type': AppConfigItem(name='session_manager.store_type', value='in_memory', type='enum', sources=[AppConfigSourceValue(source='default', value='in_memory')]),
 'llm.api_type': AppConfigItem(name='llm.api_type', value='openai', type='str', sources=[AppConfigSourceValue(source='default', value='openai')]),
 'llm.embedding_api_type': AppConfigItem(name='llm.embedding_api_type', value='openai', type='str', sources=[AppConfigSourceValue(source='default', value='openai')]),
 'llm.api_base': AppConfigItem(name='llm.api_base', value=None, type='str', sources=[AppConfigSourceValue(source='default', value=None)]),
 'llm.api_key': AppConfigItem(name='llm.api_key', value=None, type='str', sources=[AppConfigSourceValue(source='default', value=None)]),
 'llm.model': AppConfigItem(name='llm.model', value=None, type='str', sources=[AppConfigSourceValue(source='default', value=None)]),
 'llm.backup_model': AppConfigItem(name='llm.backup_model', value=None, type='str', sources=[Ap

### Simple test message

In [5]:
# Send a message to the session
user_query = "hello, what can you do?"
response_round = session.send_message(
                            user_query,
                            event_handler=lambda _type, _msg: print(f"{_type}:\n{_msg}")
                          )

init_plan:
1. Provide a brief introduction of the capabilities.
plan:
1. Provide a brief introduction of the capabilities.
current_plan_step:
Providing a brief introduction of the capabilities to the User.
send_to:
User
message:
Hello! I can assist you by breaking down tasks into subtasks and coordinating with CodeInterpreter to execute Python code to complete those tasks. This can include data analysis, file operations, and other Python-related tasks. How can I assist you today?
Planner->User:
Hello! I can assist you by breaking down tasks into subtasks and coordinating with CodeInterpreter to execute Python code to complete those tasks. This can include data analysis, file operations, and other Python-related tasks. How can I assist you today?
final_reply_message:
Hello! I can assist you by breaking down tasks into subtasks and coordinating with CodeInterpreter to execute Python code to complete those tasks. This can include data analysis, file operations, and other Python-related ta

In [6]:
# Heres what that looks like in JSON
pp.pprint(response_round.to_dict())

{'id': 'round-20231214-000218-a978c5b5',
 'post_list': [{'attachment_list': [],
                'id': 'post-20231214-000218-d987f3c1',
                'message': 'hello, what can you do?',
                'send_from': 'User',
                'send_to': 'Planner'},
               {'attachment_list': [{'content': '1. Provide a brief '
                                                'introduction of the '
                                                'capabilities.',
                                     'id': 'atta-20231214-000230-92345dcc',
                                     'type': 'init_plan'},
                                    {'content': '1. Provide a brief '
                                                'introduction of the '
                                                'capabilities.',
                                     'id': 'atta-20231214-000230-6e218e27',
                                     'type': 'plan'},
                                    {'content': 'Providing

### The classic stock market agent benchmark

> protip: make sure `llm.response_format` is `text` because when working with `json` i found it crashed when using the code interpreter.
> 💡 Up to 11/30/2023, the json_object and text options of `llm.response_format` is only supported by the OpenAI models later than `1106`. If you are using an older version of OpenAI model, you need to set the llm.response_format to null.

In [7]:
user_query = "Write code to plot the price of tesla stock YTD and save it to the current working directory"
response_round = session.send_message(user_query,
                                      event_handler=lambda _type, _msg: print(f"{_type}:\n{_msg}"))

![](../_starter_projects/workspace/sessions/20231212-200217-2ba37e82/cwd/tesla_stock_price_ytd.png)

### Plugins

A plugin is the equivalent to a function call in Autogen

In [8]:
import os
import pandas as pd
from injector import Injector

from taskweaver.config.config_mgt import AppConfigSource
from taskweaver.logging import LoggingModule
from taskweaver.memory import Memory, Post, Round
from taskweaver.memory.plugin import PluginModule, PluginRegistry
from taskweaver.planner import Planner
from taskweaver.plugin import Plugin, register_plugin, test_plugin

# Plugin definition
@register_plugin
class DataAnalysisPlugin(Plugin):
    def __call__(self, df: pd.DataFrame):
        """
        Performs a simple analysis on the DataFrame and returns a description.
        """
        row_count = len(df)
        column_count = len(df.columns)
        description = f"The DataFrame has {row_count} rows and {column_count} columns."
        return df, description

no registry for loading plugin


In [9]:
# Setup TaskWeaver environment
app_injector = Injector([PluginModule, LoggingModule])
app_config = AppConfigSource(config={
    "plugin.base_path": os.path.join(app_dir, "plugins"),
    "llm.api_key": "******",
})

# app.
app_injector.binder.bind(AppConfigSource, to=app_config)

plugin_registry = app_injector.get(PluginRegistry)
planner = app_injector.create_object(Planner)

# Create a memory object for the session
memory = Memory(session_id="session-1")

def send_message_to_session(query):
    # Create a new round with the user query
    round1 = Round.create(user_query=query, id="round-1")
    post1 = Post.create(message=query, send_from="User", send_to="Planner", attachment_list=[])
    round1.add_post(post1)

    # Add the round to the memory and compose the prompt
    memory.conversation.add_round(round1)
    prompt = planner.compose_prompt(rounds=memory.conversation.rounds)

    # Here you should process the prompt using the appropriate plugin or logic
    # For demonstration, I'm just printing the prompt
    print("Processing:", prompt)

    # Process the result and print the response (This is where your plugin logic will go)
    # Assuming a response is generated
    response = "Processed response for: " + query
    print("Response:\n", response)

    # Update the conversation with the response
    response_post = Post.create(message=response, send_from="Planner", send_to="User", attachment_list=[])
    round1.add_post(response_post)

    return round1

# Example usage
user_query = "Tell me about the data"
round  = send_message_to_session(user_query)

Processing: [{'role': 'system', 'content': 'You are the Planner who can coordinate CodeInterpreter to finish the user task.\n\n# The characters involved in the conversation\n\n## User Character\n- The User\'s input should be the request or additional information required to complete the user\'s task.\n- The User can only talk to the Planner.\n- The input of the User will prefix with "User:" in the chat history.\n\n## CodeInterpreter Character\n- CodeInterpreter is responsible for generating and running Python code to complete the subtasks assigned by the Planner.\n- CodeInterpreter can access the files, data base, web and other resources in the environment via generated Python code.\n- CodeInterpreter has the following plugin functions:\n\t- data_analysis: This plugin performs a basic analysis on the provided DataFrame and returns a summary.\n- CodeInterpreter can only talk to the Planner.\n- CodeInterpreter can only follow one instruction at a time.\n- CodeInterpreter returns the exec

## Comparison with Autogen

User(Human) input mode:
- Autogen ✅ (`human_input_mode` with `UserProxyAgent`)
- TaskWeaver ✅ (Planner can ask User)

Compression:
- Autogen ✅ (Compressible Agent)
- TaskWeaver ✅ (Built in compression mechanism)

Custom Agent Types:
- Autogen  ✅ (Autogen is a multi-agent framework)
- TaskWeaver ❌ (Taskweaver is single-agent framework)

Predefined Code Tools:
- Autogen  ✅ (with function calling)
- TaskWeaver  ✅ (with Plugins)

Adaptive Code Generative (for domain specific code):
- Autogen  ✅ (Technically, with context stuffing the prompt)
- TaskWeaver  ✅ (with Examples)

Session Management:
- Autogen ✅  (need to use different seed for cache)
- TaskWeaver ✅ (Sessions built in natively)

Stateful Code Execution:
- Autogen ❌ (runs .py files)
- TaskWeaver ✅ (runs stateful execution, like a jupyter notebook)

Parallel Code Execution:
- Autogen ❌ 
- TaskWeaver ✅ (If tasks are not dependent on eachother, they can run in different processes)

Code Execution Security Measures:
- Autogen ✅ (through containers)
- TaskWeaver ✅ (through predefined rules)



TaskWeaver seems excellent for when you have code dependent tasks, and appears to be a more structured way to manage those types of scenarios. <br>
Where it fails in comparison to Autogen is constraints. It's only meant to be used as a LLM coder. For my dynamic agent scenarios that would involve<br>
tasks from various roles Autogen would be a better choice. <br>

Using taskweaver as a add-on agent in a Autogen GroupChat would be very beneficial, not only in terms of code generation performance but in terms of security and maintainability also.

### Extendibility?

In [taskweaver.memory.post](https://github.com/microsoft/TaskWeaver/blob/4e9b71518c98bde20dbc87e6441ff1d84197dc48/taskweaver/memory/post.py#L13) it states that a post can only be one of these roles.
> A post is the message used to communicate between two roles. <br>
> It should always have a text_message to denote the string message,  <br>
>while other data formats should be put in the attachment. <br>
> The role can be either a User, a Planner, or a CodeInterpreter. <br>

And in `type_vars.py`:
```python
RoleName = Literal["User", "Planner", "CodeInterpreter"]
```

So it differs from autogen in that you don't create a number of agents that have different roles, with TaskWeaver it seems you use these three roles to do one specific type of task very well: **natural language to executable code**. In the paper they even state that multi-agent systems is 

>*The first approach involves one agent (powered by TaskWeaver) calling other agents via its plugins.
>Fig. 4 (a) depicts a simple example, although this can be extended to a more complex network <br> where multiple agents form a mesh network.
> he second approach involves embedding TaskWeaverpowered agents into an existing multi-agent framework, such as AutoGen [1], as demonstrated in Fig.*

### Notes in the paper:

https://export.arxiv.org/pdf/2311.17541

> TaskWeaver gets the LLM to condense tasks into larger tasks to that it doesn't go back and forward executing the code and reduces costs.

> *Our TaskWeaver is a single-agent framework that focuses on converting user requests into code, evenfor plugin calls.*

### NOTE TO SELF:

- Autogen as a plugin/example generator for TaskWeaver:
    - Plugins:
        - "I need a plugin to do ____" --> save it in Plugin store
    - Examples:
        - Domain specific code base -> llm to make examples -> store as examples